# 제 4고지 : 신경망 만들기

In [9]:
import sys
sys.path.append('..')

from dezero import Variable, Parameter
from dezero import optimizers
from dezero.utils import *
from tools import plot_2dfunc
from dezero.models import MLP
import dezero.functions as F
import dezero.layers as L

import numpy as np
import math
import matplotlib.pyplot as plt

## step38 : 형상 변환 함수

transpose 함수는 단순히 2차원 행렬을 전치시키는 것 뿐만 아니라 형상의 인덱스 위치를 변경할 수 있다.

In [2]:
x = Variable(np.random.randn(2, 3, 4))

y = x.transpose(1, 2, 0)
print(y.shape)
print(y)

(3, 4, 2)
variable([[[-0.31664991  0.05976221]
           [-1.16458472 -0.66508818]
           [ 1.38220631 -1.00647365]
           [-0.21247919  0.2440804 ]]
         
          [[-0.84664     0.20073327]
           [-0.58339933  0.31138723]
           [ 1.35237131  1.02193472]
           [ 0.12215831 -0.77173851]]
         
          [[-1.11174058  0.82383281]
           [ 1.42443297 -1.05153423]
           [-1.4322244  -0.95115303]
           [-1.210362   -0.45039212]]])


원소의 개수는 형상의 숫자를 모두 곱한 값과 같다. 원소의 개수를 유지하는 상태에서 형상을 변화시킬 수 있다.

## step40 : 브로드캐스트 함수
> 브로드캐스트란 형상이 다른 다차원 배열끼리의 연산을 가능하게 하는 넘파이 기능이다.

dezero 프레임워크는 넘파이 기반임을 알고는 있었지만 생각보다 많이 의존한다는 생각이 들었다. <br>그러니까 프레임워크의 설계에 중점을 두고 그 외 브로드캐스트의 정확한 구현같은 것들은 넘파이의 기능으로 대체하여 간편하게 설명한다.
이번 단계에서는 브로드캐스팅 기능을 Function 클래스에 속하게 하여서 순전파와 역전파에서 사용 가능하도록 만든다.

함수에서 브로드캐스팅이 일어난다면 **"브로드캐스팅 -> 함수 계산"** 순서일 것이다. 따라서 역전파의 구현 순서는 기존 함수 구조에서 역전파 계산 이후<br>
순선파시의 브로드캐스팅 여부 확인 후 형상 조정, 즉 **"기울기 계산 -> sum_to"** 이다.

## step42 : 선형 회귀
> 회귀란 x값에 대한 실수값 y를 찾는 과정, 방법이다.

In [3]:
# 토이 데이터셋
np.random.seed(0)
x = np.random.rand(100, 1)
y = 5 + 2 * x + np.random.rand(100, 1)
x, y = Variable(x), Variable(y)         # 생략 가능

W = Variable(np.zeros((1, 1)))
b = Variable(np.zeros(1))

def predict(x):
    y = F.matmul(x, W) + b
    return y

def mean_squared_error(x0, x1):
    diff = x0 - x1
    return F.sum(diff ** 2) / len(diff)

lr = 0.1
iters = 101

for i in range(iters):
    y_pred = predict(x)
    loss = mean_squared_error(y, y_pred)

    W.cleargrad()
    b.cleargrad()
    loss.backward()

    W.data -= lr * W.grad.data      # data 속성은 반드시 ndarray 인스턴스이어야 한다. 따라서 데이터 갱신 때 계산 그래프가
    b.data -= lr * b.grad.data      # 업데이트되지 않는다.
    if i % 50 == 0:
        print(W, b, loss)

variable([[0.64433458]]) variable([1.29473389]) variable(42.296340129442335)
variable([[2.28830557]]) variable([5.37981674]) variable(0.08694585483964615)
variable([[2.11563939]]) variable([5.46732269]) variable(0.07901006311507543)


위의 mse함수는 메모리에 diff와 그것의 제곱,그리고 합한 값까지 총 세개의 추가적인 메모리를 낭비하게 된다.<br>
그치만, 이를 class화 시켜서 함수로 만든다면 계산그래프는 3개의 노드에서 1개의 노드로 줄 것이다.

### 오류 해결
> Linear 함수의 순전파를 dezero 함수로 구성할 경우 ndarray로 들어온 data들을 전부 Variable객체로 변환하게 된다.<br>
이렇게 되면 Variable객체의 data 또한 Variable객체가 되어서 이후 연산 수행 시 새로 생기는 변수가 Variable객체인 상태로 Variable클래스의 초기화 함수에 들어가 오류를 일으킨다.

## step43 : 신경망

In [4]:
np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

# 가중치 초기화
I, H, O = 1, 10, 1
W1 = Variable(0.01 * np.random.randn(I, H))
b1 = Variable(np.zeros(H))
W2 = Variable(0.01 * np.random.randn(H, O))
b2 = Variable(np.zeros(O))

# 신경망 추론
def predict(x):
    y = F.linear(x, W1, b1)
    y = F.sigmoid(y)
    y = F.linear(y, W2, b2)
    return y

lr = 0.2
iters = 1000

# 신경망 학습
for i in range(iters):
    y_pred = predict(x)
    loss = F.mean_squared_error(y, y_pred)

    W1.cleargrad()
    b1.cleargrad()
    W2.cleargrad()
    b2.cleargrad()
    loss.backward()

    W1.data -= lr * W1.grad.data
    b1.data -= lr * b1.grad.data
    W2.data -= lr * W2.grad.data
    b2.data -= lr * b2.grad.data
    
    if i % 1000 == 0:
        print(loss)

variable(0.8473695850105871)


위 모양과 라이브러리를 사용할 때의 모양이 점점 닮아진다.<br>
그리고 I, H, O는 각각 입력, 은닉, 출력 층의 차원 수를 의미한다.<br>
매개변수 초기화는 0이 아닌 임의의 값을 사용하는 것을 확인할 수 있다.

## step44 : 매개변수를 모아두는 계층

In [5]:
layer = L.Layer()

layer.p1 = Parameter(np.array(1))
layer.p2 = Parameter(np.array(2))
layer.p3 = Variable(np.array(3))
layer.p4 = 'test'

print(layer._params)
print('-'*60)

for name in layer._params:
    print(name, layer.__dict__[name])

{'p1', 'p2'}
------------------------------------------------------------
p1 variable(1)
p2 variable(2)


In [6]:
np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

l1 = L.Linear(10)   # 출력 크기 지정
l2 = L.Linear(1)

def predict(x):
    y = l1(x)
    y = F.sigmoid(y)
    y = l2(y)
    return y

lr =  0.2
iters = 10000

for i in range(iters):
    y_pred = predict(x)
    loss = F.mean_squared_error(y, y_pred)

    l1.cleargrads()
    l2.cleargrads()
    loss.backward()

    for l in [l1, l2]:
        for p in l.params():
            p.data -= lr * p.grad.data

    if i % 1000 == 0:
        print(loss)

variable(0.8165178479901415)
variable(0.2499028014603371)
variable(0.24609874026436834)
variable(0.23721586110833612)
variable(0.20793217994822144)
variable(0.12311919860580511)
variable(0.0788816839034867)
variable(0.0765607529785731)
variable(0.0764336464779914)
variable(0.07619374494842987)


## step45 : 계층을 모아두는 계층

In [7]:
model = L.Layer()
model.l1 = L.Linear(5)
model.l2 = L.Linear(3)

def predict(model, x):
    y = model.l1(x)
    y = F.sigmoid(y)
    y = model.l2(y)
    return y

for p in model.params():        # 순전파 실행을 하지 않아서 가중치는 None이다.
    print(p)

model.cleargrads()

variable([0. 0. 0.])
variable(None)
variable([0. 0. 0. 0. 0.])
variable(None)


## step47 : 소프트맥스 함수와 교차 엔트로피 오차

In [8]:
model = MLP((10, 3))
x = np.array([[0.2, -0.4], [0.3, 0.5], [1.3, -3.2], [2.1, 0.3]])
t = np.array([2, 0, 1, 0])
y = model(x)
loss = F.softmax_cross_entropy_simple(y, t)

print(loss)

variable(1.1784240974937457)


## step48 : 다중 클래스 분류

In [ ]:
max_epoch = 300
batch_size = 30
hidden_size = 10
lr = 1.0

x, t = dezero.datasets.Spiral(train=True)
model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)

data_size = len(x)
max_iter = math.ceil(data_size / batch_size)    # 소수점 반올림

for epoch in range(max_epoch):
    index = np.random.permutation(data_size)
    sum_loss = 0

    for i in range(max_iter):
        batch_index = index[i * batch_size:(i + 1) * batch_size]
        batch_x = x[batch_index]
        batch_t = t[batch_index]

        y = model(batch_x)
        loss = F.softmax_cross_entropy_simple(y, batch_t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(batch_t)
    if epoch % 30 == 0:
        avg_loss = sum_loss / data_size
        print('epoch %d, loss %.2f' % (epoch + 1, avg_loss))
avg_loss = sum_loss / data_size
print('epoch %d, loss %.2f' % (epoch + 1, avg_loss))

epoch 1, loss 1.13
epoch 31, loss 0.71
epoch 61, loss 0.64
epoch 91, loss 0.46
epoch 121, loss 0.36
epoch 151, loss 0.25
epoch 181, loss 0.20
epoch 211, loss 0.17
epoch 241, loss 0.15
epoch 271, loss 0.14
epoch 300, loss 0.13


## step49 : Dataset 클래스와 전처리
> __getitem__ 과 __len__ 이 정의되어야 한다.<br>
큰 용량의 데이터에 대해서는 미리 메모리상에 모든 데이터를 올리는게 아니라 배치 단위의 데이터만 불러온다.


DataLoader는 __next__ 와 __iter__ 를 통해 미니배치 단위로 데이터셋을 나눠준다.

## step50 : 미니배치를 뽑아주는 DataLoader

N개의 피쳐에 대한 1개의 정답 -> shape이 다를 수 있다.